<a href="https://colab.research.google.com/github/Sanjeeda-Jeba/Brain_Tumor_Classification/blob/main/BrainTumorClassification_updated.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Dataset
 * Link: `https://www.kaggle.com/datasets/ahmedhamada0/brain-tumor-detection`

In [ ]:
# Toget the dataset saving from DropBox

!wget https://www.dropbox.com/scl/fi/h9m24hojzfumn0utj9c9s/Br35H_Dataset.zip?rlkey=tpflcfx3tchdpnblxnuxkj6ze&dl=0


In [ ]:
# UNZIP

!unzip /content/Br35H_Dataset.zip?rlkey=tpflcfx3tchdpnblxnuxkj6ze


In [21]:
import cv2
import os
import tensorflow as tf
from tensorflow import keras
from PIL import Image
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from keras.utils import normalize, to_categorical
from keras.models import Sequential, load_model
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense

In [4]:
img_dir = "/content/Dataset/"

In [5]:
no_img = os.listdir(img_dir + "no/")
yes_img = os.listdir(img_dir + "yes/")
len(no_img)
len(yes_img)

### Creating two lists:
input_size = 64
d_s = []
label = []


In [6]:

for i, img_name in enumerate(no_img):
  if(img_name.split('.')[1] == 'jpg'):
    img = cv2.imread(img_dir + "no/" + img_name)
    img = Image.fromarray(img , 'RGB')
    img = img.resize((input_size,input_size))
    d_s.append(np.array(img))
    label.append(0)

In [7]:
#Now The same exact steps for the Yes folder:
for i, img_name in enumerate(yes_img):
  if(img_name.split('.')[1] == 'jpg'):
    img = cv2.imread(img_dir + "yes/" + img_name)
    img = Image.fromarray(img , 'RGB')
    img = img.resize((input_size,input_size))
    d_s.append(np.array(img))
    label.append(1)


In [8]:
len(d_s)

3000

In [9]:
### Now turning both the lists (d_s and label) into numpy arrays:

dataset = np.array(d_s)

label = np.array(label)

In [10]:
### Spliting the dataset:

x_train, x_test, y_train, y_test = train_test_split(dataset, label, test_size = 0.2,
                                                    random_state = 0)

In [11]:
### Now normalize the data:
x_train = normalize(x_train, axis = 1)

x_test = normalize(x_test, axis = 1)

# Model Building

In [12]:
model = Sequential()

In [13]:
model.add(Conv2D(32, (3,3), input_shape = (input_size, input_size, 3)))

model.add(Activation('relu'))

model.add(MaxPooling2D(pool_size=(2,2)))

In [14]:
model.add(Conv2D(32, (3,3), kernel_initializer= 'he_uniform'))

model.add(Activation('relu'))

model.add(MaxPooling2D(pool_size=(2,2)))

In [15]:
model.add(Conv2D(64, (3,3), kernel_initializer= 'he_uniform'))

model.add(Activation('relu'))

model.add(MaxPooling2D(pool_size=(2,2)))

In [16]:
model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1))        #Binary Classification
model.add(Activation('sigmoid'))

* Binary CrossEntropy =1 (activation: `sigmoid`)
* Categorical CrossEntropy = 2, (activation: `softMax`)



In [17]:
model.compile(loss = 'binary_crossentropy', optimizer= 'adam',
              metrics=['accuracy'])

In [56]:
model.fit(x_train, y_train, batch_size=16, verbose=1, epochs = 20,
          validation_data= (x_test, y_test), shuffle= False)

Epoch 1/20
150/150 [==============================] - 1s 6ms/step - loss: 0.0372 - accuracy: 0.9875 - val_loss: 0.1825 - val_accuracy: 0.9767
Epoch 2/20
150/150 [==============================] - 1s 6ms/step - loss: 0.0242 - accuracy: 0.9892 - val_loss: 0.2770 - val_accuracy: 0.9617
Epoch 3/20
150/150 [==============================] - 1s 6ms/step - loss: 0.0058 - accuracy: 0.9983 - val_loss: 0.2295 - val_accuracy: 0.9717
Epoch 4/20
150/150 [==============================] - 1s 6ms/step - loss: 0.0014 - accuracy: 0.9996 - val_loss: 0.2312 - val_accuracy: 0.9717
Epoch 5/20
150/150 [==============================] - 1s 5ms/step - loss: 5.5448e-04 - accuracy: 1.0000 - val_loss: 0.2476 - val_accuracy: 0.9733
Epoch 6/20
150/150 [==============================] - 1s 6ms/step - loss: 3.7202e-04 - accuracy: 1.0000 - val_loss: 0.2595 - val_accuracy: 0.9733
Epoch 7/20
150/150 [==============================] - 1s 5ms/step - loss: 2.8757e-04 - accuracy: 1.0000 - val_loss: 0.2644 - val_accuracy: 0

In [57]:
model.save('BrainTumor20EpochsB16.keras')

# Prediction

In [58]:
MODEL = load_model('BrainTumor20EpochsB16.keras')

In [65]:
pred_img = cv2.imread('/content/pred/pred5.jpg') #Copy the path of any image from pred folder

img = Image.fromarray(pred_img)
img = img.resize([64,64])

### numpy array

img = np.array(img)

input_img = np.expand_dims(img, axis=0)

prediction = model.predict(input_img)

1/1 [==============================] - 0s 26ms/step


In [66]:
prediction

array([[1.]], dtype=float32)

In [67]:

if prediction[0] == 1:
  print('Tumor detected!')

else:
  print('Congratulations! no tumor detected!')

Tumor detected!
